In [1]:
import torch
cuda_num = str(5)
device_s = "cuda:" + cuda_num
device = torch.device(device_s if torch.cuda.is_available() else "cpu")

In [33]:
def read_list(text_path):
    lsit = []
    with open('%s' % text_path, 'r', encoding="utf8") as f:  # 打开一个文件只读模式
        line = f.readlines()  # 读取文件中的每一行，放入line列表中
        for line_list in line:
            lsit.append(line_list.replace('\n', ''))
    return lsit

import ast
def read_list2(text_path):
    lsit=[]
    with open('%s' % text_path, 'r', encoding="utf8") as f:  # 打开一个文件只读模式
        line = f.readlines()  # 读取文件中的每一行，放入line列表中
        for line_list in line:
            line = ast.literal_eval(line_list.replace('\n', ''))
            lsit.append(line)
    return lsit


def store_list(lsit, text_path):
    ff = open(text_path, encoding='utf-8', mode='w')
    for line_list in lsit:
        ff.write(str(line_list))  # 写入一个新文件中
        ff.write("\n")


def get_stop_words():
    # 加载停用词
    stop_words_path = "../../stopwords0.txt"
    stopwords = set([item.strip() for item in open(stop_words_path, 'r').readlines()])
    return stopwords
    # print(stopwords)


# 去掉文本中的停用词
def drop_stopwords(stopwords, line):
    line_clean = []
    for word in line:
        if word =='':
            continue
        if word in stopwords:
            continue
        line_clean.append(word)
    return line_clean

def get_st():
    return read_list("../../st1.txt")
    
    
    
def clear(line):
    line_clean = []
    for word in line:
        if not word.isdigit():
            line_clean.append(word)
    return line_clean

In [3]:
import sys
st = get_st()
data_list0 = read_list('./text.txt')
# data_list0=data_list0[0:1]
data_list=[]
i_n=0
for i in data_list0:
    sys.stdout.write(str(i_n) + '\r')
    i_n=i_n+1
    for j in st:#去除标点符号
        i=i.replace(j,'')
    data_list.append(i)
# print(data_list)

In [4]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased").to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# inputs = tokenizer("Hello world!", return_tensors="pt")
# outputs = model(**inputs)

In [6]:
def tokenize_function(example):
    return tokenizer(example,return_tensors="pt",padding=True,truncation=True)

tokenized_datasets = [tokenize_function(i) for i in data_list]

In [12]:
print(len(tokenized_datasets))

20808


In [29]:
modeled_datasets=[]
i_n=0

for i in range(len(tokenized_datasets)):
    sys.stdout.write(str(i_n) + '\r')
    i_n=i_n+1
    try0=tokenized_datasets[i].to(device)
    out0 = model(**try0)
    # print(outputs[1][0].tolist())
    # print(len(outputs[1][0].tolist()))
    modeled_datasets.append(out0[1][0].tolist())

In [30]:
store_list(modeled_datasets, 'modeled_datasets.txt')

In [32]:
print(len(modeled_datasets))
print(len(modeled_datasets[0]))

20808
768


In [34]:
modeled_datasets0=read_list2('modeled_datasets.txt')

In [35]:
print(len(modeled_datasets0))
print(len(modeled_datasets0[0]))

20808
768


In [9]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(tokenized_datasets, batch_size=128)

In [8]:

for batch in train_dataloader:
    batch=batch.to(device)
    outputs = model(**batch)
    break



RuntimeError: stack expects each tensor to be equal size, but got [1, 512] at entry 0 and [1, 472] at entry 1

In [41]:
# from transformers import squad_convert_examples_to_features
# features = squad_convert_examples_to_features(
#     examples=data_list,  tokenizer=tokenizer,max_seq_length=512,doc_stride)

TypeError: squad_convert_examples_to_features() missing 4 required positional arguments: 'max_seq_length', 'doc_stride', 'max_query_length', and 'is_training'

In [31]:
# from transformers import glue_convert_examples_to_features as convert_examples_to_features
# data_features = convert_examples_to_features(data_list,tokenizer,label_list=None,output_mode="classification")

TypeError: 'NoneType' object is not iterable

In [38]:
print(len(tokenized_datasets))
print(len(tokenized_datasets[0]))
print(len(tokenized_datasets[0][0]))

20808
3
512


In [25]:
print(len(outp[0][0]))
print(outp[0][0])#该文章每个词

512
tensor([[-0.6944,  0.2582, -0.0607,  ..., -0.4951,  0.5002,  0.3865],
        [-0.2278,  0.0936, -0.6565,  ...,  0.5138,  1.1057, -0.1172],
        [-0.5605,  0.2595,  0.5565,  ..., -0.2362, -0.6021,  0.1308],
        ...,
        [-0.8794, -0.6774, -0.0228,  ...,  0.1449, -0.1038,  0.5325],
        [-0.1562, -0.3481, -0.1050,  ..., -0.0992,  0.4419, -0.1524],
        [ 0.3078,  0.3086,  0.1337,  ..., -0.2235, -0.4089,  0.2020]],
       device='cuda:5', grad_fn=<SelectBackward0>)


In [24]:
print(len(outp[0][0][0]))#某个词
print(outp[0][0][0])

768
tensor([-6.9438e-01,  2.5816e-01, -6.0673e-02, -3.1512e-01, -3.2401e-01,
        -1.3490e-02,  3.5672e-01,  6.8995e-01, -2.7142e-01,  1.7811e-01,
        -1.3229e-01, -6.1978e-01, -9.6737e-01,  1.2803e-01,  3.0904e-02,
         8.3929e-02, -2.9550e-01,  3.1537e-01,  2.8993e-01,  3.0545e-01,
         5.3507e-02, -5.4684e-01,  2.3491e-01,  1.5954e-01,  6.7469e-01,
        -6.0988e-01,  3.5565e-02, -6.9016e-01, -5.2587e-01, -2.8850e-01,
         5.5732e-01,  1.2682e-01, -2.5702e-02,  3.8728e-02,  5.2750e-01,
        -1.1681e-01,  8.0179e-01, -8.1869e-02,  4.4787e-01,  2.1115e-01,
        -3.1938e-01,  6.7282e-01,  1.1077e-01,  7.6984e-02,  9.7705e-02,
        -9.2495e-02, -3.2289e+00, -2.0712e-01, -2.0023e-01, -1.7386e-01,
        -1.3848e-01, -1.1753e-02,  5.3485e-01, -4.5319e-02,  6.6432e-01,
         6.3904e-01, -8.5435e-01, -2.2076e-01,  2.3065e-01, -1.7449e-01,
         1.9379e-01,  7.6357e-02, -2.2030e-01,  1.7953e-02,  2.2015e-01,
        -1.7139e-02, -2.3261e-01,  6.0209e-01, 

In [26]:
print(len(outp[1][0]))
print(outp[1][0])#该文章

768
tensor([-0.7637, -0.5992, -0.9091,  0.6618,  0.6772, -0.3101,  0.2112,  0.5965,
        -0.7950, -1.0000, -0.6371,  0.9519,  0.9778,  0.3979,  0.7420, -0.6960,
        -0.1789, -0.6718,  0.5241,  0.6210,  0.6425,  1.0000, -0.3876,  0.6070,
         0.5447,  0.9773, -0.7986,  0.8754,  0.8854,  0.6707, -0.4393,  0.6491,
        -0.9861, -0.4332, -0.9233, -0.9926,  0.6707, -0.4966, -0.1304, -0.3725,
        -0.5985,  0.5542,  1.0000, -0.2291,  0.6128, -0.5382, -1.0000,  0.4731,
        -0.6534,  0.8636,  0.7010,  0.9590,  0.4876,  0.5122,  0.6915, -0.7283,
         0.0044,  0.2450, -0.5036, -0.7297, -0.6876,  0.5927, -0.8735, -0.7612,
         0.8628,  0.8201, -0.6175, -0.5027, -0.3123,  0.1124,  0.6817,  0.5490,
        -0.6130, -0.6185,  0.7043,  0.4654, -0.7679,  1.0000, -0.2923, -0.9639,
         0.9553,  0.8460,  0.7228, -0.4939,  0.3926, -1.0000,  0.5922, -0.3953,
        -0.9769,  0.5823,  0.6463, -0.4163,  0.8649,  0.7490, -0.3615, -0.6988,
        -0.4211, -0.8904, -0.5597, -